## USE pretrained network to output keypoint's description

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from munkres import Munkres

from descriptor_CNN3 import DesNet
model = DesNet()
if args.cuda:
    model.cuda()
weight_path = "checkpoint_b.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])

# load patches
patches_a_dir = "../keypoint_detector/patches-a.pt"
patches_i_dir = "../keypoint_detector/patches-i.pt"
patches_q_dir = "../keypoint_detector/patches-q.pt"
patches_a = torch.load(patches_a_dir)
patches_i = torch.load(patches_i_dir)
patches_q = torch.load(patches_q_dir)

print(patches_a.shape)
print(patches_i.shape)
print(patches_q.shape)
patches_a =  patches_a.view(-1, 1, 32, 32).cuda()
patches_i =  patches_i.view(-1, 1, 32, 32).cuda()
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

#description = model(patches)
with torch.no_grad():
    description_a = model(patches_a)
    description_i = model(patches_i)
    description_q = model(patches_q)

description_a = description_a.view(-1, 30, 128)
description_i = description_i.view(-1, 30, 128)
description_q = description_q.view(-1, 30, 128)
print(description_a.shape)
print(description_i.shape)
print(description_q.shape)

print("Done!")

torch.Size([175, 30, 1, 32, 32])
torch.Size([140, 30, 1, 32, 32])
torch.Size([35, 30, 1, 32, 32])
torch.Size([175, 30, 128])
torch.Size([140, 30, 128])
torch.Size([35, 30, 128])
Done!


In [55]:
similarity_matrix_one_to_one = np.zeros((35,140))
#print(description_q[0].shape)
#print(description_i[0].shape)
hungarian = Munkres()
for i in range(0,35):
    top = 0
    for j in range(0,140):
        dist = np.zeros((30,30))
        for k in range(0,30):
            for l in range(0,30):
                dist[k][l] = np.linalg.norm(np.subtract(description_q[i][k].cpu().numpy(), description_i[j][l].cpu().numpy()))
        cost_indecies = hungarian.compute(np.copy(dist))
        tmp = np.zeros(len(cost_indecies))
        for k in range(0,len(cost_indecies)):
            tmp[k] = np.exp(-(dist[cost_indecies[k][0]][cost_indecies[k][1]]))
        similarity_matrix_one_to_one[i][j]=tmp.sum()
        if(top < similarity_matrix_one_to_one[i][j]):
            top = similarity_matrix_one_to_one[i][j]
            print((i,j))
print(similarity_matrix_one_to_one)
# Check point: saving the tensor to prevent re computing it
save = False
if save:
    print("Saving a checkpoint...")
    torch.save(similarity_matrix_one_to_one, "checkpoint-1-to-1.pt")
print("Done!")


(0, 0)


KeyboardInterrupt: 

In [2]:
# start from here to reload the tensor
similarity_matrix_one_to_one = torch.load("checkpoint-1-to-1.pt")
print(similarity_matrix_one_to_one.shape)
print(similarity_matrix_one_to_one)
print("Done!")

(35, 140)
[[14.72182043 14.66791591 14.02765892 ...  8.35000678  8.51899126
   8.39069858]
 [ 8.64536997  8.60855022  8.54037473 ...  8.52345117  8.77902
   8.36236534]
 [ 8.89078664  9.01640916  8.75485121 ...  8.22393745  8.17894583
   8.48824189]
 ...
 [ 8.18620794  8.3762208   8.16903619 ...  8.3156659   8.25351963
   8.67568428]
 [ 7.98199662  8.26353275  7.93197186 ...  7.88224947  7.84024087
   8.1232731 ]
 [ 8.66657365  8.48269225  8.54763821 ...  9.60727395 16.61573185
  10.99162673]]
Done!


In [147]:
similarity_matrix_many_to_many = np.zeros((35,140))
threshold = 0.1
print(description_q.shape)
print(description_i.shape)
for i in range(0,35):
    top = [np.NINF,(0,0)]
    for j in range(0,140):
        dist = np.zeros((30))
        for k in range(0,30):
            tmp = np.zeros(30)
            for l in range(k,30):
                query = description_q[i][k].cpu().numpy()
                image = description_i[j][l].cpu().numpy()
                tmp[k] = np.sum(np.absolute(np.subtract(query,image)))
            dist[k] = np.sum(tmp)
        s = np.zeros(30)
        x = np.zeros(30)
        x_s = np.zeros(30)
        for k in range (0,30):
            s[k] = np.exp(-(dist[k]))
        lamda = np.sqrt(1 / np.dot(np.transpose(s), s))
        norm = np.linalg.norm(s,axis=0)
        x = np.true_divide(s, norm)
        x_s = np.copy(x)
        x_s[x_s > threshold] = 1
        x_s[x_s < threshold] = 0
        a = np.dot(np.transpose(s), x_s)
        b = lamda * (np.dot(np.transpose(x_s), x_s) - 1)
        similarity_matrix_many_to_many[i][j]= a - b
        #print("lamda: "+str(lamda))
        #print("norm: "+str(norm))
        #print("s: "+str(s))
        #print("x: "+str(x))
        #print("x_s: "+str(x_s))
        #print("a: "+str(a))
        #print("b: "+str(b))
        #print("Similarity: " + str(similarity_matrix_many_to_many[i][j]))
        if (similarity_matrix_many_to_many[i][j] > top[0]):
            top = [similarity_matrix_many_to_many[i][j],(i,j)]
            if j != 0:
                print(str(top[0]) + ": " + str(top[1]))
    print(top)
    print("_____")
save = True
if save:
    print("Saving a checkpoint...")
    torch.save(similarity_matrix_many_to_many, "checkpoint-many-to-many.pt")
print("Done!")

torch.Size([35, 30, 128])
torch.Size([140, 30, 128])
0.037225084216548555: (0, 1)
[0.037225084216548555, (0, 1)]
_____
-5064.7322806420325: (1, 1)
-120.97887898205002: (1, 4)
0.00010408860412707332: (1, 50)
0.00033175584724198727: (1, 54)
[0.00033175584724198727, (1, 54)]
_____
-30471.71899137429: (2, 1)
-9410.458563806607: (2, 2)
-1523.4581479403366: (2, 11)
[-1523.4581479403366, (2, 11)]
_____
-156984.0553783118: (3, 1)
-6874.774325072158: (3, 2)
0.00011067152930159248: (3, 3)
0.0031412035338172625: (3, 13)
[0.0031412035338172625, (3, 13)]
_____
-279588.94170314254: (4, 1)
-70611.50734630339: (4, 2)
-3232.5204821461252: (4, 18)
0.0001307567213167708: (4, 47)
[0.0001307567213167708, (4, 47)]
_____
-353933.15969248663: (5, 1)
-127419.90140797339: (5, 3)
-29131.487497198395: (5, 13)
8.063189522430217e-05: (5, 16)
0.00013751743176277983: (5, 55)
[0.00013751743176277983, (5, 55)]
_____
-8416.461703004728: (6, 1)
[-8416.461703004728, (6, 1)]
_____
-3745.45357381842: (7, 2)
[-3745.453573818

In [149]:
# Import many to many matching here and skip last step
similarity_matrix_many_to_many = torch.load("checkpoint-many-to-many.pt")
print(similarity_matrix_many_to_many.shape)
print(similarity_matrix_many_to_many)
print("Done!")

(35, 140)
[[-2.98336963e+05  3.72250842e-02 -1.00635046e+02 ... -2.78449602e+05
  -5.10215228e+05 -2.62080047e+05]
 [-2.81131723e+05 -5.06473228e+03 -6.90652123e+05 ... -5.92564613e+05
  -8.04762518e+05 -1.18169516e+05]
 [-4.37792635e+05 -3.04717190e+04 -9.41045856e+03 ... -2.43625613e+05
  -8.60235095e+05 -7.15324510e+05]
 ...
 [-2.82578483e+05 -1.46529936e+06 -3.51561808e+05 ... -9.19246002e+05
  -1.29164416e+06 -5.56886524e+05]
 [-1.13040041e+06 -1.76930444e+05 -4.81211451e+04 ... -1.77692153e+06
  -1.07944688e+06 -1.16543249e+06]
 [-2.26861573e+05 -3.21144812e+05 -1.82440256e+05 ... -1.61619189e+05
   9.63903388e-03 -1.81803341e+04]]
Done!


Recall = $TP(\hat{X})/ 4$
Percision= $TP(\hat{X})/ k\in {1..4}$